In [ ]:
from authenta import AuthentaClient

client = AuthentaClient(
    base_url="https://platform.authenta.ai",
    client_id="YOUR_CLIENT_ID",
    client_secret="YOUR_CLIENT_SECRET",
)

### Method reference

#### `upload_file(path: str, model_type: str) -> Dict[str, Any]`
- Two-step upload helper.
- Calls `POST /media` with `name`, `contentType`, `size`, `modelType` to obtain `mid` and `uploadUrl`, then `PUT` the file to S3 using that URL.
- Returns the media JSON from `POST /media` (includes at least `mid`, `status`, `modelType`, timestamps, and possibly URLs).

#### `wait_for_media(mid: str, interval: float = 5.0, timeout: float = 600.0) -> Dict[str, Any]`
- Polls `GET /media/{mid}` until `status` is one of `PROCESSED`, `FAILED`, or `ERROR`, or until `timeout` seconds have elapsed.
- Sleeps `interval` seconds between polls.
- Returns the final media JSON or raises `TimeoutError` if it times out.

#### `list_media(**params) -> Dict[str, Any]`
- Wraps `GET /media`
- Returns all media in Authenta

#### `process(path: str, model_type: str, interval: float = 5.0, timeout: float = 600.0) -> Dict[str, Any]`
- High-level convenience method.
- Internally calls `upload_file` to create and upload the media, then `wait_for_media` with the returned `mid`.[file:3]
- Returns the final processed media JSON (including flags like `fake`, confidence scores, and `resultURL` / `heatmapURL` if present).

#### `delete_media(mid: str) -> None`
- Wraps `DELETE /media/{mid}`
- Deletes the given media record 


In [ ]:
# 1. Upload and wait for result

media = client.process("samples/nano_img.png", model_type="AC-1")
print(media["status"], media.get("fake"), media.get("resultURL"))

In [ ]:
# 2. Upload only 

upload_meta = client.upload_file("samples/nano_img.png", model_type="AC-1")
print(upload_meta["mid"], upload_meta["status"])

In [ ]:
# 3. Process from known mid
final_media = client.wait_for_media(upload_meta["mid"])

In [ ]:
# 4. List all media (mid)

all_media = client.list_media()
print(all_media)

In [ ]:
# 5. Delete a media

client.delete_media(upload_meta["mid"])

### Save Heatmaps for Images/Videos 

In [ ]:
save_heatmap(media=media, out_path="results/image_heatmap.jpg", model_type="AC-1")

In [ ]:
media2 = client.process("data_samples/test_00000121.mp4", model_type="DF-1")

save_heatmap_video(media2, "results")


In [ ]:
save_bounding_box_video(media2, src_video_path="data_samples/test_00000121.mp4", out_video_path="results/analyzed_video.mp4")
